# Trabalho de IA - Usando Machine Learning

Sistemas de Informação - UNISUL

- Alexandre Ventura
- Mateus Wanderlinde
- Júlia Staub

## House Price Prediction - Predição de preço de casas

A previsão dos preços dos imóveis está se tornando cada vez mais importante e benéfica. Os preços dos imóveis são um bom indicador tanto da condição geral do mercado quanto da saúde econômica de um país.

### Lendo os dados

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
# Importando a biblioteca pandas para tratamento dos dados
import pandas as pd

# Lendo os dados e criando um Dataframe, chamado "dados"
dados = pd.read_csv('/home/alexandre-ventura/trabalho_ia/data.csv')

In [43]:
# Visualizando as informações do nosso Dataframe
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4600 non-null   float64
 2   bedrooms       4600 non-null   float64
 3   bathrooms      4600 non-null   float64
 4   sqft_living    4600 non-null   int64  
 5   sqft_lot       4600 non-null   int64  
 6   floors         4600 non-null   float64
 7   waterfront     4600 non-null   int64  
 8   view           4600 non-null   int64  
 9   condition      4600 non-null   int64  
 10  sqft_above     4600 non-null   int64  
 11  sqft_basement  4600 non-null   int64  
 12  yr_built       4600 non-null   int64  
 13  yr_renovated   4600 non-null   int64  
 14  street         4600 non-null   object 
 15  city           4600 non-null   object 
 16  statezip       4600 non-null   object 
 17  country        4600 non-null   object 
dtypes: float

Portanto o dataframe usado, contém as seguintes colunas: 

* **date:** Data da venda da casa.
* **price:** Preço de venda da casa.
* **bedrooms:** Número de quartos.
* **bathrooms:** Número de banheiros.
* **sqft_living:** Área interna da casa em pés quadrados.
* **sqft_lot:** Área do terreno em pés quadrados.
* **floors:** Número de andares na casa.
* **waterfront:** Indicador se a casa tem vista para a água.
* **view:** Classificação da vista da casa.
* **condition:** Classificação da condição da casa.
* **sqft_above:** Área acima do solo em pés quadrados.
* **sqft_basement:** Área do porão em pés quadrados.
* **yr_built:** Ano de construção da casa.
* **yr_renovated:** Ano de renovação da casa.
* **street:** Rua da casa.
* **city:** Cidade onde fica a casa.
* **statezip:** Código postal do estado.
* **country:** País onde a casa está.

Ao todo são 18 colunas, mas será que é necessário usar todas para fazer a predição?

Vamos analisar algumas colunas do dataframe que é possível desconsiderar.

### Limpeza e tratamento dos dados

In [44]:
# Pegando os todos os valores da coluna "country"
dados['country'].value_counts()

country
USA    4600
Name: count, dtype: int64

Levando em conta que todas as casas estão no EUA, podemos apagar essa coluna pois ela não influenciara na nossa predição.

In [45]:
# Apagando a coluna "Country" do nosso Dataframe
dados.drop(['country'], axis=1, inplace=True)

Podemos também apagar a coluna "street".

In [46]:
# Apagando a coluna "street" do nosso Dataframe
dados.drop(['street'], axis=1, inplace=True)

In [47]:
# Apagando a coluna "statezip" do nosso Dataframe
dados.drop(['statezip'], axis=1, inplace=True)

E também a coluna "date".

In [48]:
dados.drop(['date'], axis=1, inplace=True)

In [49]:
# Visualizando o dataframe atualizando.
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          4600 non-null   float64
 1   bedrooms       4600 non-null   float64
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   int64  
 4   sqft_lot       4600 non-null   int64  
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   int64  
 7   view           4600 non-null   int64  
 8   condition      4600 non-null   int64  
 9   sqft_above     4600 non-null   int64  
 10  sqft_basement  4600 non-null   int64  
 11  yr_built       4600 non-null   int64  
 12  yr_renovated   4600 non-null   int64  
 13  city           4600 non-null   object 
dtypes: float64(4), int64(9), object(1)
memory usage: 503.3+ KB


Agora que foram excluidas ambas as colunas, vamos verificar se existe algum valor nulo em nosso dataset.

In [50]:
dados.isnull().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
city             0
dtype: int64

Não temos valores nulos, o que é bom e nos poupa trabalho. 

Mas vamos olhar com mais calma algumas colunas. Importante destacar que todo o dataset está com medidas em inglês.
É interessante convertemos alguns valores para medidas internacionais, para melhor interpretação e visualização dos dados.


In [51]:
# Transformando a área interna da casa, de pés quadrados, para metros quadrados. 
dados['area_interna'] = (dados['sqft_living'] * 0.0929).round()

Vamos fazer isso com todas as colunas que apresentam a medida "pés quadrados".

In [52]:
# Transformando a área do terreno, de pés quadrados, para metros quadrados.
dados['area_terreno'] = (dados['sqft_lot'] * 0.0929).round()

In [53]:
# Transformando a área acima do solo, de pés quadrados, para metros quadrados.
dados['area_acima_solo'] = (dados['sqft_above'] * 0.0929).round()

In [54]:
# Transformando a área do porão, de pés quadrados, para metros quadrados.
dados['area_porao'] = (dados['sqft_basement'] * 0.0929).round()

Agora que todas as medidas de área foram transformadas para metros quadrados, vamos olhar a nova configuração
das colunas, e remover as colunas modificadas.

In [55]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            4600 non-null   float64
 1   bedrooms         4600 non-null   float64
 2   bathrooms        4600 non-null   float64
 3   sqft_living      4600 non-null   int64  
 4   sqft_lot         4600 non-null   int64  
 5   floors           4600 non-null   float64
 6   waterfront       4600 non-null   int64  
 7   view             4600 non-null   int64  
 8   condition        4600 non-null   int64  
 9   sqft_above       4600 non-null   int64  
 10  sqft_basement    4600 non-null   int64  
 11  yr_built         4600 non-null   int64  
 12  yr_renovated     4600 non-null   int64  
 13  city             4600 non-null   object 
 14  area_interna     4600 non-null   float64
 15  area_terreno     4600 non-null   float64
 16  area_acima_solo  4600 non-null   float64
 17  area_porao    

In [56]:
# Excluindo as colunas que contém a medida "pés quadrados"
dados.drop(['sqft_living'], axis=1, inplace=True)
dados.drop(['sqft_lot'], axis=1, inplace=True)
dados.drop(['sqft_above'], axis=1, inplace=True)
dados.drop(['sqft_basement'], axis=1, inplace=True)

In [57]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            4600 non-null   float64
 1   bedrooms         4600 non-null   float64
 2   bathrooms        4600 non-null   float64
 3   floors           4600 non-null   float64
 4   waterfront       4600 non-null   int64  
 5   view             4600 non-null   int64  
 6   condition        4600 non-null   int64  
 7   yr_built         4600 non-null   int64  
 8   yr_renovated     4600 non-null   int64  
 9   city             4600 non-null   object 
 10  area_interna     4600 non-null   float64
 11  area_terreno     4600 non-null   float64
 12  area_acima_solo  4600 non-null   float64
 13  area_porao       4600 non-null   float64
dtypes: float64(8), int64(5), object(1)
memory usage: 503.3+ KB


Esse é o novo conjunto de colunas que temos em nosso dataframe:

* **price:** Preço de venda da casa.
* **bedrooms:** Número de quartos.
* **bathrooms:** Número de banheiros.
* **floors:** Número de andares na casa.
* **waterfront:** Indicador se a casa tem vista para a água.
* **view:** Classificação da vista da casa.
* **condition:** Classificação da condição da casa.
* **yr_built:** Ano de construção da casa.
* **yr_renovated:** Ano de renovação da casa.
* **city:** Cidade onde fica a casa.
* **statezip:** Código postal do estado.
* **area_interna:** Área interna da casa em metros quadrados.
* **area_terreno:** Área do terreno em metros quadrados.
* **area_acima_solo:** Área acima do solo em metros quadrados.
* **area_porao:** Área do porão em metros quadrados.

Se olharmos com atenção para nosso dataset ainda vamos encontrar algumas coisas que aparentemente não faz muito sentido. Por exemplo nas colunas "bathrooms" e "floors", como uma casa pode ter 1.75 banheiros? Ou 2.5 andares?

Por isso decidimos arredondar para números inteiros, antes de começarmos nossa analise exploratória. 

In [58]:
# Visualizando quantos banheiros cada casa possui.
dados['bathrooms'].value_counts()

bathrooms
2.50    1189
1.00     743
1.75     629
2.00     427
2.25     419
1.50     291
2.75     276
3.00     167
3.50     162
3.25     136
3.75      37
4.50      29
4.00      23
4.25      23
0.75      17
4.75       7
5.00       6
5.25       4
5.50       4
1.25       3
0.00       2
6.25       2
5.75       1
8.00       1
6.50       1
6.75       1
Name: count, dtype: int64

In [59]:
# Arredondando a quantidade de banheiros em números inteiros.
dados['bathrooms'] = dados['bathrooms'].round()

In [60]:
# Visualizando quantos banheiros cada casa possui, agora em números inteiros. 
dados['bathrooms'].value_counts()

bathrooms
2.0    2955
1.0     763
3.0     579
4.0     274
5.0      17
6.0       8
0.0       2
8.0       1
7.0       1
Name: count, dtype: int64

Já atualizamos os valores de "bathrooms", vamos atualizar os valores de "floors".

In [61]:
# Arredondando a quantidade de andares em números inteiros.
dados['floors'] = dados['floors'].round()

In [62]:
# Visualizando quantos andares cada casa possui, agora em números inteiros. 
dados['floors'].value_counts()

floors
2.0    2296
1.0    2174
3.0     128
4.0       2
Name: count, dtype: int64

Ainda podemos melhorar alguns dados, por exemplo, ao invés do ano de construção, mostrar quantos anos cada casa possui. 

In [63]:
# Importando uma biblioteca que nos ajudará a trabalhar com datas
from datetime import datetime

dados["idade_casa"] = datetime.today().year - dados["yr_built"]

In [64]:
dados.drop(['yr_built'], axis=1, inplace=True)

In [65]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            4600 non-null   float64
 1   bedrooms         4600 non-null   float64
 2   bathrooms        4600 non-null   float64
 3   floors           4600 non-null   float64
 4   waterfront       4600 non-null   int64  
 5   view             4600 non-null   int64  
 6   condition        4600 non-null   int64  
 7   yr_renovated     4600 non-null   int64  
 8   city             4600 non-null   object 
 9   area_interna     4600 non-null   float64
 10  area_terreno     4600 non-null   float64
 11  area_acima_solo  4600 non-null   float64
 12  area_porao       4600 non-null   float64
 13  idade_casa       4600 non-null   int64  
dtypes: float64(8), int64(5), object(1)
memory usage: 503.3+ KB


### Machine Learning

Vamos partir para a escolha do modelo de treinamento de máquina, e avaliação do mesmo modelo.
Mas antes vamos preparar ainda mais os dados para o treinamento. 

In [66]:
# Criar novas features

# importando a bibiloteca numpy para manipulação matemáticas. 
import numpy as np

# Criando uma nova coluna
dados['idade_apos_renovacao'] = np.where(dados['yr_renovated'] > 0, 
                                         datetime.today().year - dados['yr_renovated'], 
                                         dados['idade_casa'])

In [67]:
# Removendo coluna
dados.drop(['yr_renovated'], axis=1, inplace=True)

É interessante que removamos os Outliers. Outliers são valores muito diferentes da maioria dos dados, como se fossem "fora da curva". Eles podem ser muito altos ou muito baixos em relação ao restante do conjunto de dados.

Aqui iremos remover outliers (valores muito altos) na coluna price, mantendo apenas os 97% das casas mais baratas.

In [68]:
import numpy as np

# Definir um limite superior baseado no percentil 97% dos preços das casas
limite_superior = np.percentile(dados['price'], 97)

# Filtrar os dados, removendo as casas que possuem preços acima do limite
dados_filtrados = dados[dados['price'] < limite_superior]

# Exibir quantas casas foram removidas
print(f"Casas removidas por outliers: {dados.shape[0] - dados_filtrados.shape[0]}")
dados = dados[dados['price'] < limite_superior]

""" # Remover outliers nos preços (acima do percentil 97%)
limite_superior = np.percentile(dados['price'], 97)
dados = dados[dados['price'] < limite_superior] """

Casas removidas por outliers: 138


" # Remover outliers nos preços (acima do percentil 97%)\nlimite_superior = np.percentile(dados['price'], 97)\ndados = dados[dados['price'] < limite_superior] "

Vamos usar o LabelEncoder, para tratar com nossas strings. 

Esse código transforma a coluna categórica 'city' em números, permitindo que o modelo de Machine Learning a utilize.

In [69]:
from sklearn.preprocessing import LabelEncoder

# Converter colunas categóricas
label_encoder = LabelEncoder()
dados['city'] = label_encoder.fit_transform(dados['city'])

In [70]:
# Separar features e target
X = dados.drop(['price'], axis=1)
y = dados['price']

Tendo separados as features (colunas que serão usadas no treino), e o target (coluna de teste) vamos dividir os dados. Esse código divide os dados em treino e teste, permitindo que o modelo aprenda com uma parte dos dados e seja avaliado com outra parte.

In [71]:
from sklearn.model_selection import train_test_split

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Agora vamos usar o StandardScaler(), que é uma classe de pré-processamento de dados do scikit-learn. Essa classe ajuda a normalizar a escala dos dados, o que é crucial para muitos algoritmos de aprendizado de máquina que são sensíveis à variação nas escalas das características (features) dos dados.

Quando você aplica a StandardScaler() a um conjunto de dados, ela realiza as seguintes etapas para cada característica:

- Calcula a média de cada característica.
- Calcula o desvio padrão de cada característica, medindo quão dispersos estão os valores em relação à média.
- Normaliza os valores subtraindo a média e dividindo pelo desvio padrão para cada valor de característica, com a fórmula:

Este processo de normalização ajusta os dados de tal forma que a média das características transformadas é zero e o desvio padrão é um. Isso é importante porque muitos algoritmos de machine learning performam melhor quando as características estão na mesma escala.

In [72]:
# Importando o StandardScaler
from sklearn.preprocessing import StandardScaler

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Para este trabalho vamos usar dois modelos que lidam bem com problemas de regressão, como previsão de vendas.

XGBoost e LightGBM.

In [73]:
from xgboost import XGBRegressor

# Treinar o modelo XGBoost
modelo_xgb = XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
modelo_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = modelo_xgb.predict(X_test_scaled)

**XGBoost:** Imagine que você quer adivinhar o preço de uma casa. O XGBoost funciona como um time de especialistas que fazem previsões, aprendendo com os erros uns dos outros. Ele começa com uma previsão simples e, a cada rodada, melhora corrigindo os erros anteriores. Esse método, chamado boosting, faz com que o modelo fique mais preciso a cada passo. É ótimo para encontrar padrões escondidos, mas pode ser um pouco mais lento em grandes volumes de dados.

In [74]:
import warnings
warnings.filterwarnings("ignore")

from lightgbm import LGBMRegressor

# Treinar o modelo LightGBM
modelo_lgb = LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
modelo_lgb.fit(X_train_scaled, y_train)
y_pred_lgb = modelo_lgb.predict(X_test_scaled)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1195
[LightGBM] [Info] Number of data points in the train set: 3569, number of used features: 12
[LightGBM] [Info] Start training from score 502331.273114


**LightGBM:** O LightGBM faz algo parecido, mas de forma mais rápida e eficiente. Em vez de crescer todas as árvores da mesma forma, ele foca primeiro nas partes mais importantes, acelerando o aprendizado. Isso faz com que funcione muito bem em grandes conjuntos de dados, sem precisar de tanto tempo ou memória do computador. Porém, por ser muito agressivo na busca por padrões, pode ser um pouco mais sensível a erros.

<hr>

### Avaliação do Modelo: MAE e RMSE

#### MAE (Mean Absolute Error) - Erro Médio Absoluto
O **MAE** mede a média dos erros absolutos entre os valores reais e os previstos. Ele indica, em média, o quanto nossas previsões estão erradas.  
- Quanto **menor o MAE**, melhor o modelo.  
- Representa o erro médio em unidades monetárias.  



#### RMSE (Root Mean Squared Error) - Raiz do Erro Quadrático Médio
O **RMSE** calcula o erro médio, mas dando mais peso para erros grandes. Ele é mais sensível a outliers do que o MAE.  
- Quanto **menor o RMSE**, melhor o modelo.  
- Penaliza erros grandes mais do que o MAE.  



### Diferença entre MAE e RMSE
|  | **MAE** | **RMSE** |
|---|---|---|
| **Interpretação** | Erro médio absoluto | Penaliza erros grandes |
| **Sensibilidade a outliers** | Baixa | Alta |
| **Uso** | Se erros grandes não forem tão críticos | Se erros grandes devem ser evitados |

Para escolher entre MAE e RMSE, depende do objetivo do modelo. Se queremos um erro médio mais interpretável, usamos MAE. Se queremos um modelo que evite grandes desvios, usamos RMSE. 🚀


In [75]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Avaliação do XGBoost
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb) ** 0.5
print(f'MAE XGBoost: {mae_xgb:.2f}')
print(f'RMSE XGBoost: {rmse_xgb:.2f}')

MAE XGBoost: 103800.24
RMSE XGBoost: 164945.88


In [76]:
# Avaliação do LightGBM
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
rmse_lgb = mean_squared_error(y_test, y_pred_lgb) ** 0.5
print(f'MAE LightGBM: {mae_lgb:.2f}')
print(f'RMSE LightGBM: {rmse_lgb:.2f}')

MAE LightGBM: 105277.82
RMSE LightGBM: 164486.64


### Por que MAE e RMSE são importantes?

Avaliar um modelo de Machine Learning é essencial para garantir que suas previsões sejam precisas e úteis. **MAE e RMSE nos ajudam a entender o quão próximos os valores previstos estão dos valores reais.**

### Principais Motivos para Calcular MAE e RMSE  

- **Medição da Qualidade do Modelo:**  
Essas métricas indicam o **erro médio das previsões**, permitindo saber se o modelo está funcionando bem ou precisa de ajustes.  

- **Comparação entre Modelos:**  
Se testamos diferentes algoritmos (XGBoost, LightGBM, etc.), podemos comparar os erros e escolher o melhor.  

- **Ajuste de Hiperparâmetros:**  
Usamos essas métricas para verificar **se as alterações no modelo melhoram ou pioram seu desempenho**.  

**Evitar Problemas como Overfitting e Underfitting**  
- Se o erro for **muito alto**, o modelo pode não ter aprendido o suficiente (**underfitting**).  
- Se o erro no treino for muito menor do que no teste, o modelo pode estar **decorando os dados** (**overfitting**).  

**Tomada de Decisão**  
Se o erro médio é de **$100.000**, sabemos que o modelo pode estar **subestimando ou superestimando os preços** e podemos considerar ajustes.  

**Conclusão:** MAE e RMSE são essenciais para avaliar e aprimorar modelos de Machine Learning, garantindo que suas previsões sejam mais confiáveis. 


In [77]:
preco_medio = dados["price"].mean()
print(f'Preço médio das casas: ${preco_medio:,.2f}')


Preço médio das casas: $499,859.71


### Fazendo o algoritmo prever o preço.

Por fim, vamos fazer com que o algoritmo preva o preço das casas, e faremos isso com os dois modelos diferentes. 

In [78]:
# Criando um dicionário com os dados de um novo imóvel
novo_imovel = {
    'city': 'Seattle', 
    'bedrooms': 3,
    'bathrooms': 2,
    'floors': 1,
    'waterfront': 0,
    'view': 2,
    'condition': 4,
    'idade_casa': 20,
    'area_interna': 150,
    'area_terreno': 300,
    'area_acima_solo': 120,
    'area_porao': 30,
    'idade_apos_renovacao': 25
}

# Convertendo a cidade para o valor numérico usando LabelEncoder
novo_imovel['city'] = label_encoder.transform([novo_imovel['city']])[0]

# Convertendo para um DataFrame
novo_imovel_df = pd.DataFrame([novo_imovel])

# Fazer a previsão com o modelo treinado
previsao = modelo_xgb.predict(novo_imovel_df)

print(f'Preço previsto para o novo imóvel: ${previsao[0]:,.2f}')


Preço previsto para o novo imóvel: $960,084.12


O modelo XGboost, preveu que uma casa, com aqueles dados que inserimos vale $960,084.12

In [79]:
# Criando um dicionário com os dados de um novo imóvel
novo_imovel = {
    'city': 'Seattle', 
    'bedrooms': 3,
    'bathrooms': 2,
    'floors': 1,
    'waterfront': 0,
    'view': 2,
    'condition': 4,
    'idade_casa': 20,
    'area_interna': 150,
    'area_terreno': 300,
    'area_acima_solo': 120,
    'area_porao': 30,
    'idade_apos_renovacao': 25
}

# Convertendo a cidade para o valor numérico usando LabelEncoder
novo_imovel['city'] = label_encoder.transform([novo_imovel['city']])[0]

# Convertendo para um DataFrame
novo_imovel_df = pd.DataFrame([novo_imovel])

# Fazer a previsão com o modelo treinado
previsao = modelo_lgb.predict(novo_imovel_df)

print(f'Preço previsto para o novo imóvel: ${previsao[0]:,.2f}')


Preço previsto para o novo imóvel: $957,879.85


Com os mesmos dados o modelo LightGBM previu que a casa é avaliada em $957,879.85

<hr>

### Avaliação dos Modelos

### Resultados:

#### XGBoost:
- **MAE:** $103,800 → 20,8% do preço médio  
- **RMSE:** $164,945 → 32,9% do preço médio  

#### LightGBM:
- **MAE:** $105,277 → 21,0% do preço médio  
- **RMSE:** $164,486 → 32,9% do preço médio  



### O que isso significa?
- O **erro médio absoluto (MAE)** indica que, em média, o modelo erra o preço da casa em cerca de **$103K a $105K**.  
- O **RMSE** sugere que os erros maiores (**outliers**) podem estar impactando o modelo, pois o valor está acima de **30% do preço médio**.  
- Como o erro representa cerca de **20-21% do preço médio**, o modelo já tem um desempenho razoável, mas pode melhorar.  



### Possíveis Melhorias:
1️**Feature Engineering:** Criar novas variáveis, como **densidade populacional da cidade** ou **preço médio por metro quadrado**.  
2️ **Tratar Outliers:** Como o **RMSE é bem maior que o MAE**, pode haver **outliers influenciando o modelo**. Podemos testar a remoção de valores extremos no preço.  
3️ **Ajuste de Hiperparâmetros:** Testar **GridSearchCV** ou **Optuna** para encontrar os melhores parâmetros para XGBoost e LightGBM.  
4️ **Modelos Ensemble:** Criar um modelo que **combine XGBoost e LightGBM** para aproveitar o melhor de ambos.  

**Conclusão:** O modelo já apresenta um bom desempenho, mas ainda há espaço para melhorias!  
